In [ ]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    first_in : bool
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
# # 处理栈中数据
# def Process_data(stack):
#     some_data = []
#     while(stack):
#         info_pop = stack.pop()
#         if(type(info_pop) != Node and info_pop[0] == "{"):
#             break
#         elif(type(info_pop) != Node and info_pop[0] == "}"):
#             continue
#         else:
#             some_data.append(info_pop)
#     print(some_data)
#     now_node = Node(
#         name = None,
#         offset = [],
#         channels = {},
#         child = [],
#         parent = None,
#         first_in = True
#     )
#     last_node = None
#     for i in some_data:
#         # if(type(i) != Node and i[0] == "OFFSET"):
#         #     now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         # elif(type(i) != Node  and i[0] == "CHANNELS"):
#         #     now_node.channels = {
#         #         "num_channels" : int(i[1]),
#         #         "channels" : [i[1:]]
#         #     }
#         # elif(type(i) != Node and i[0] == "JOINT"):
#         #     now_node.name = i[1]
#         # elif(type(i) == Node):
#         #     last_node = i
#         #     i += 1
#         if(type(i) == Node):
#             last_node = i
#             continue
#         if(type(i) != Node and i[0] == "OFFSET"):
#             now_node.offset = [float(i[1]), float(i[2]), float(i[3])]
#         elif(type(i) != Node and i[0] == "CHANNELS"):
#             now_node.channels = {
#                 "num_channels" : int(i[1]),
#                 "channels" : [i[1:]]}
#         elif(type(i) != Node and i[0] == "End" or i[0] == "Joint"):
#             if(i[0] == "End") : last_node.name = i[0] + i[1]
#             if(i[0] == "Joint") : last_node.name = i[1]
#             last_node.parent = now_node.name
#             now_node.child.append(last_node)
#             last_node.first_in = False
#             joint_List.append(last_node)
            
#         print(now_node)
#         print(last_node)
#     return now_node
        
    
            
len_info = len(info_List)
node_list = []
stack = deque()
# for i in range(len_info):
#     if(info_List[i][0] == "{"):
#         stack.append(info_List[i])
#         while(info_List[i][0] != "}"):
#             i += 1
#             stack.append(info_List[i])
#     elif(info_List[i][0] == "}"):
#         stack.pop()
#         print(stack.pop())
for i in range(len_info):
    if info_List[i][0] == "ROOT":
        root = Node(
             name = info_List[i][0] + info_List[i][1],
                offset = [],
                channels = {},
                child = [],
                parent = None,
                first_in = True)
        node_list.append(root)
        joint_List.append(root)
        continue
    if info_List[i][0] == "{" or info_List[i][0] == "OFFSET" or \
        info_List[i][0] == "CHANNELS" or info_List[i][0] == "End" or \
        info_List[i][0] == "JOINT":
            stack.append(info_List[i])
            continue
    if info_List[i][0] == "}":
        stack.append(info_List[i])

 
       
    
        


In [ ]:
info_List

In [ ]:
node_list

In [ ]:
class Node:
    def __init__(self, name):
        self.name = name
        self.offset = None
        self.channels = []
        self.children = []

    def __repr__(self):
        return f"Node({self.name})"


def build_skeleton(bvh_data):
    stack = []
    root = None
    current_node = None

    for line in bvh_data:
        if line[0] == 'ROOT' or line[0] == 'JOINT':
            # 创建新节点
            node = Node(line[1])
            if current_node is not None:
                # 添加为当前节点的子节点
                current_node.children.append(node)
                stack.append(current_node)
            else:
                root = node
            current_node = node
        elif line[0] == 'OFFSET':
            # 设置偏移量
            current_node.offset = tuple(map(float, line[1:]))
        elif line[0] == 'CHANNELS':
            # 设置通道信息
            current_node.channels = line[2:]  # 跳过通道数量
        elif line[0] == '}':
            # 结束当前节点的定义，返回到父节点
            if stack:
                current_node = stack.pop()
        elif line[0] == 'End' and line[1] == 'Site':
            # 处理末端节点
            end_site_node = Node('End Site')
            current_node.children.append(end_site_node)
            stack.append(current_node)
            current_node = end_site_node

    return root

# 解析示例数据
bvh_data = [['ROOT', 'Hips'], ['{'], ['OFFSET', '0.00', '0.00', '0.00'], ['CHANNELS', '6', 'Xposition', 'Yposition', 'Zposition', 'Xrotation', 'Yrotation', 'Zrotation']]
skeleton = build_skeleton(info_List)
print(skeleton.children)


In [ ]:
from dataclasses import dataclass
from collections import deque

@dataclass
class Node:
    name : str
    offset : list
    channels : dict
    child : list
    parent : str
    ID : int
    type : str
    
file_path = "walk60.bvh"
bvh_data = []
with open(file_path, 'r') as f:
    for line in f:
        bvh_data.append(line.split())

num_frames = 0
frame_time = 0
frames = []
joint_List = []
info_List = []
for line in bvh_data:
    this_line = line
    if(this_line[0] == "Hierarchy"): continue
    if(this_line[0] == "MOTION"): continue
    if(this_line[0] == "Frames:"):
        num_frames = int(this_line[1])
        continue
    if(this_line[0] == "Frame"):
        frame_time = float(this_line[2])
        continue
    if(this_line[0] == "{" or
       this_line[0] == "ROOT" or
       this_line[0] == "}" or
       this_line[0] == "End" or
       this_line[0] == "JOINT" or
       this_line[0] == "OFFSET" or
       this_line[0] == "CHANNELS"):
        info_List.append(this_line)
        continue
    frames.append(this_line)
    
offset_stack = deque()
channel_stack = deque()
node_stack = deque()
joint_List = []
node_id = 0

for info in info_List:
    if(info[0] == "ROOT"):
        node_stack.append(Node(
            name = "RootJoint",
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "root"
        ))
        node_id += 1
        continue
    elif info[0] == "{": continue
    elif info[0] == "OFFSET":
        offset_stack.append(list(map(float, info[1:])))
        continue
    elif info[0] == "CHANNELS":
        channel_stack.append({
            "channel_num" : int(info[1]),
            "channels" : info[2:]
        })
    elif info[0] == "JOINT":
        node_stack.append(Node(
            name = info[1],
            offset = None,
            channels = None,
            child = [],
            parent = None,
            ID = node_id,
            type = "joint"
        ))
        node_id += 1
        continue
    elif info[0] == "End":
        node_stack.append(Node(
            name = info[0] + info[1],
            offset = None,
            channels = None,
            parent = None,
            child = None,
            type = "End",
            ID = node_id
        ))
        node_id += 1
        continue
    elif info[0] == "}":
        pop_node = node_stack.pop()
        if pop_node.type == "End":
            top_node = node_stack[-1]
            pop_node.name = top_node.name + "_end"
            pop_node.parent = top_node.name
            pop_node.child = None
            offset_top = offset_stack.pop()
            pop_node.offset = offset_top
            pop_node.channels = None
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "joint":
            top_node = node_stack[-1]
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = top_node.name
            top_node.child.append(pop_node)
            joint_List.append(pop_node)
        elif pop_node.type == "root":
            pop_offset = offset_stack.pop()
            pop_channels = channel_stack.pop()
            # print(pop_channels)
            pop_node.offset = pop_offset
            pop_node.channels = pop_channels
            pop_node.parent = None
            joint_List.append(pop_node)
        continue
               
            

In [ ]:
for node in joint_List:
    if(node.name == "RootJoint"):
        print(len(node.child))

In [ ]:
from read_bvh import Bone_Tree

bone_tree = Bone_Tree('walk60.bvh')

In [ ]:
bone_tree.all_channels

In [ ]:
bone_tree.channels

In [ ]:
bone_tree.frames[0]

In [ ]:
from read_bvh import Bone_Tree

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R
def load_motion_data(bvh_file_path):
    """part2 辅助函数，读取bvh文件"""
    with open(bvh_file_path, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            if lines[i].startswith('Frame Time'):
                break
        motion_data = []
        for line in lines[i+1:]:
            data = [float(x) for x in line.split()]
            if len(data) == 0:
                break
            motion_data.append(np.array(data).reshape(1,-1))
        motion_data = np.concatenate(motion_data, axis=0)
    return motion_data

In [ ]:
motion_data = load_motion_data(bvh_file_path= 'walk60.bvh')

In [ ]:
motion_data

In [ ]:
print(motion_data.shape)

In [ ]:
motion_data_my = Bone_Tree('walk60.bvh').frames

In [ ]:
motion_data_my

In [ ]:
from read_bvh import Bone_Tree
tree = Bone_Tree(bvh_file_path= 'walk60.bvh')

In [ ]:
tree.sorted_joint_list

In [ ]:
name_list = []
id_list = []
parent_list = []
offset_list = []
for joint in tree.sorted_joint_list:
    name_list.append(joint.name)
    id_list.append(joint.ID)
    parent_list.append(joint.parent["parent_id"])
    offset_list.append(joint.offset)
print(name_list)
print(id_list)
print(parent_list)
print(len(offset_list))
print(tree.node_count)

In [ ]:
tree.joint_List

In [ ]:
from read_bvh import Bone_Tree
tree = Bone_Tree(bvh_file_path= 'walk60.bvh')

In [ ]:
print(tree.joint_name)
print(tree.)
print(tree.joint_parent)

In [ ]:
from read_bvh import Bone_Tree
from dataclasses import dataclass
from collections import deque
import numpy as np
from scipy.spatial.transform import Rotation as R

bone_tree = Bone_Tree("walk60.bvh")

In [ ]:
joint_List = bone_tree.sorted_joint_list
joint_name = bone_tree.joint_name
joint_parent = bone_tree.joint_parent
joint_offset = bone_tree.joint_offset
motion_data = bone_tree.frames


joint_List

In [ ]:
def find_parent_rotation(Q, parent_index):
    for joint in Q:
        if joint['id'] == parent_index:
            return joint['rotation']
        
def find_parent_location(Q, parent_index):
    for joint in Q:
        if joint['id'] == parent_index:
            return joint['location']

def forward_kinematics(
        joint_list: list,
        motion_data: list,
    ) -> np.array:
        # Q4 = Q3R4 = Q2R3R4 = Q1R2R3R4 = Q0R1R2R3R4 = R0R1R2R3R4
        # DP 累推
        # return : joint_position : shape -> num_frames * num_joint * 3
        # return : joint_rotation : shape -> num_frames * num_joint * 4 (quaternion)
        joint_all_frames_info_rotation_local = []
        for frame in motion_data:
            # Q : this_frame joint_rotation -> shape : num_joint * 4
            Q = []
            # Only root joint has translation
            T0 = None
            # joint_list is sorted by ID : the sequence follow the bvh hierarchy
            for joint in joint_list:
                if joint.type == "root":
                    # root channels is 6 : first 3 is translate , second 3 is rotation
                    translate = frame[:3]
                    rotation = frame[3:6]
                    # translate_euler = R.from_euler('XYZ', translate, degrees=True)
                    T0 = np.array(translate)
                    rotation_euler = R.from_euler("XYZ", rotation, degrees=True)
                    rotation_quaternion = rotation_euler.as_quat()
                    # root's location is (0,0,0)
                    Q.append(
                        {
                            "id": 0,
                            "rotation" : rotation_quaternion,
                            "location" : np.array([0, 0, 0]),
                        }
                    )
                elif joint.type == "joint":
                    joint_index = joint.ID - 1
                    parent_index = joint.parent["parent_id"]
                    joint_rotation = frame[
                       joint.channel_index : joint.channel_index + 3
                    ]
                    #print(joint.ID)
                    #print(joint_index)
                    joint_rotation = np.array(list(map(float, joint_rotation)))
                    # parent_roation is quaternion
                    parent_rotation = find_parent_rotation(Q, parent_index)
                    #print(joint_rotation)
                    #print(type(joint_rotation))
                    #print(R.from_euler("xyz", joint_rotation, degrees=True))
                    #print(R.from_euler("xyz", joint_rotation, degrees=True).as_matrix())
                    this_joint_rotation = R.from_matrix(
                        R.from_quat(parent_rotation).as_matrix()
                        @ R.from_euler("xyz", joint_rotation, degrees=True).as_matrix()
                    ).as_quat()
                    # print(this_joint_rotation)
                    # print("--------------")
                    # parent_location is np.array -> 3 dim vec
                    parent_location = find_parent_location(Q, parent_index)
                    # joint_offset is np.array -> 3 dim vec
                    joint_offset = joint.offset
                    # ID == 1 -> need translate
                    if joint.ID == 1:
                        T0 = list(map(float, T0))
                        this_joint_location = (
                            parent_location
                            + R.from_quat(parent_rotation).apply(np.array(joint_offset))
                            + T0
                        )
                    else:
                        this_joint_location = (
                            parent_location + R.from_quat(parent_rotation).apply(np.array(joint_offset))
                        )
                    Q.append(
                        {
                            'id' : joint.ID,
                            "rotation" : this_joint_rotation,
                            "location" : this_joint_location,
                        }
                    )
                elif joint.type == "End":
                    parent_index = joint.parent["parent_id"]
                    parent_rotation = find_parent_rotation(Q, parent_index)
                    parent_location = find_parent_location(Q, parent_index)
                    this_offset = joint.offset
                    this_joint_location = parent_location + R.from_quat(parent_rotation).apply(np.array(this_offset))
                    Q.append(
                        {
                            "id" : joint.ID,
                            "rotation" : None,
                            "location" : this_joint_location
                        }
                    )
            joint_all_frames_info_rotation_local.append(Q)
        return joint_all_frames_info_rotation_local
                
                    

In [ ]:
joint_all_position = forward_kinematics(
    joint_List,
    motion_data = motion_data)
print(np.array(joint_all_position).shape)

In [ ]:
print(joint_all_position[0])

In [1]:
from read_bvh import Bone_Tree
from dataclasses import dataclass
from collections import deque
import numpy as np
from scipy.spatial.transform import Rotation as R

bone_tree = Bone_Tree("walk60.bvh")

In [6]:
bone_tree.all_frame_location[0].shape



(25, 3)

In [4]:
bone_tree.all_frame_rotation.shape

(182, 25, 4)